In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
#특징점 찾기
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable, for performance improvement
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
#특징점에 점 그리기 (화면에)
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
#특징점 추출, nparray로 저장 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [6]:
####################파라미터 값 설정 칸###########################
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
#############################수어 단어 입력칸#########################
actions = np.array(['bank', 'number', 'password', 'period', 'online banking', 'limit', 'change',
                   'decrease', 'increase', 'value', 'branch office', 'registration card', 'bank book', 'stock', 'deal',
                   'savings', 'register', 'family', 'receive', 'photo',  
                    'income', 'property', 'cash', 'interest', 'credit card', 'loan',
                    'exchange', 'withdraw', 'pay back', 'nothing'])

# Thirty videos worth of data
################단어 당 100개 영상 촬영 #######################
no_sequences = 100

# Videos are going to be 30 frames in length
#################비디오 당 프레임 수######################
sequence_length = 30

# Folder start
start_folder = 30

In [7]:
###############폴더 생성#############

for action in actions:
    try: 
        os.makedirs(os.path.join(DATA_PATH, str(action)))
    except:
        pass


for action in actions: 
    dirmax = 0
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

In [18]:
#데이터셋 촬영(한단어씩 촬영)
cap = cv2.VideoCapture(0)
# Set mediapipe model 
escape = False   
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP

        # Loop through sequences aka videos
    for sequence in range(no_sequences):
    # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)
                
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
              # Show to screen
                cv2.imshow('OpenCV Feed', image)
                
            # NEW Export keypoints
            ################### 여기 빨간색에 단어 변경하면 촬영할 단어 변경됨, 저장 폴더도 자동으로 변경 #########
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, 'nothing', str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            if cv2.waitKey(10) & 0xFF == ord('w'):
                escape = True
                break 
        if escape == True:
            break
        

cap.release()
cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
X = np.array(sequences)
X.shape

(3000, 30, 258)

In [9]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
log_dir = os.path.join('Logs')
callback = EarlyStopping(monitor='loss', patience=20)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=150, callbacks=[callback])

Epoch 1/150
85/85 [==============================] - 7s 40ms/step - loss: 3.2986 - categorical_accuracy: 0.0696
Epoch 2/150
85/85 [==============================] - 3s 39ms/step - loss: 2.7995 - categorical_accuracy: 0.1385
Epoch 3/150
85/85 [==============================] - 3s 39ms/step - loss: 2.2636 - categorical_accuracy: 0.2652
Epoch 4/150
85/85 [==============================] - 3s 40ms/step - loss: 1.7614 - categorical_accuracy: 0.4063
Epoch 5/150
85/85 [==============================] - 3s 40ms/step - loss: 1.4086 - categorical_accuracy: 0.5141
Epoch 6/150
85/85 [==============================] - 3s 40ms/step - loss: 1.0841 - categorical_accuracy: 0.6337
Epoch 7/150
85/85 [==============================] - 3s 40ms/step - loss: 1.6811 - categorical_accuracy: 0.4844
Epoch 8/150
85/85 [==============================] - 3s 40ms/step - loss: 1.0563 - categorical_accuracy: 0.6378
Epoch 9/150
85/85 [==============================] - 3s 40ms/step - loss: 0.7618 - categorical_accuracy:

85/85 [==============================] - 4s 43ms/step - loss: 0.2064 - categorical_accuracy: 0.9315
Epoch 74/150
85/85 [==============================] - 4s 43ms/step - loss: 0.1265 - categorical_accuracy: 0.9633
Epoch 75/150
85/85 [==============================] - 4s 43ms/step - loss: 0.0561 - categorical_accuracy: 0.9837
Epoch 76/150
85/85 [==============================] - 4s 44ms/step - loss: 0.0666 - categorical_accuracy: 0.9807
Epoch 77/150
85/85 [==============================] - 4s 44ms/step - loss: 0.3530 - categorical_accuracy: 0.9289
Epoch 78/150
85/85 [==============================] - 4s 43ms/step - loss: 0.0524 - categorical_accuracy: 0.9833
Epoch 79/150
85/85 [==============================] - 4s 43ms/step - loss: 0.0325 - categorical_accuracy: 0.9911
Epoch 80/150
85/85 [==============================] - 4s 43ms/step - loss: 0.0125 - categorical_accuracy: 0.9978
Epoch 81/150
85/85 [==============================] - 4s 43ms/step - loss: 0.0616 - categorical_accuracy: 0.9

In [27]:
res = model.predict(X_test)
actions[np.argmax(res[5])]
actions[np.argmax(y_test[5])]

10/10 [==============================] - 1s 20ms/step


'bank book'

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print("test 데이터 정확도 :" , accuracy)

10/10 [==============================] - 1s 22ms/step - loss: 0.1303 - categorical_accuracy: 0.9833
test 데이터 정확도 : 0.9833333492279053


In [13]:
model.save('action4.h5')

In [14]:
import tensorflow as tf
model = tf.keras.models.load_model('action4.h5')

In [15]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6
capture_state = False
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        if cv2.waitKey(10) & 0xFF == ord('w'):
            capture_state = True
        if capture_state == True:
            if len(sequence) == 0:
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(1000)
                
            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
           

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 3: 
                    sentence = sentence[-3:]
                capture_state = False
                sequence = []

                # Viz probabilities


        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image,' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 1s 507ms/step
decrease
1/1 [==============================] - 0s 25ms/step
interest
1/1 [==============================] - 0s 29ms/step
decrease
1/1 [==============================] - 0s 26ms/step
decrease
1/1 [==============================] - 0s 28ms/step
number
1/1 [==============================] - 0s 25ms/step
password
1/1 [==============================] - 0s 26ms/step
period
1/1 [==============================] - 0s 27ms/step
online banking
1/1 [==============================] - 0s 27ms/step
decrease
1/1 [==============================] - 0s 25ms/step
limit
1/1 [==============================] - 0s 26ms/step
limit
1/1 [==============================] - 0s 26ms/step
change
1/1 [==============================] - 0s 27ms/step
nothing
1/1 [==============================] - 0s 25ms/step
increase
